In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from reservoirpy import ESN
import psutil
import threading
import time

In [21]:
df = pd.read_csv('SWaT.csv')
df.columns = df.columns.str.strip()
df['Normal/Attack'] = (df['Normal/Attack'] == 'Attack').astype(int)
X_raw = df.drop(['Timestamp', 'Normal/Attack'], axis=1)
y = df['Normal/Attack']

scaler = MinMaxScaler()
X_raw = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, stratify=y, random_state=42
)

In [22]:
corr_matrix = X_train.corrwith(y_train).abs()
top_10_corr = corr_matrix.nlargest(10).index
X_train_corr = X_train[top_10_corr]
X_test_corr = X_test[top_10_corr]
X_train_clean = X_train_corr.loc[:, X_train_corr.std() > 0]
X_test_clean = X_test_corr[X_train_clean.columns]
X_train_clean = X_train_clean.T.drop_duplicates().T
X_test_clean = X_test_clean[X_train_clean.columns]
y_train_arr = np.array(y_train.values).reshape(-1, 1)

C:\Users\chand\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\chand\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [23]:
chosen_reservoir = 10
chosen_spectral_radius = 0.1
chosen_input_scaling = 1.0

In [24]:
process = psutil.Process()
cpu_usages = []

def cpu_logger(stop_event, interval=0.05):
    while not stop_event.is_set():
        cpu_usages.append(process.cpu_percent(interval=None))
        time.sleep(interval)

In [25]:
cpu_stop = threading.Event()
cpu_thread = threading.Thread(target=cpu_logger, args=(cpu_stop, 0.05))
cpu_usages.clear()

num_cores = psutil.cpu_count(logical=True)
print(f"Your system has {num_cores} logical CPU cores.\n")

cpu_thread.start()
start = time.time()
esn = ESN(
    units=chosen_reservoir,
    sr=chosen_spectral_radius,
    input_scaling=chosen_input_scaling,
    ridge=1e-6,
    seed=42
)
esn = esn.fit(X_train_clean.values, y_train_arr)
train_time = time.time() - start
cpu_stop.set()
cpu_thread.join()

Your system has 8 logical CPU cores.



In [27]:
cpu_stop = threading.Event()
cpu_thread = threading.Thread(target=cpu_logger, args=(cpu_stop, 0.05))
cpu_usages.clear()

num_cores = psutil.cpu_count(logical=True)
print(f"Your system has {num_cores} logical CPU cores.\n")

cpu_thread.start()
start = time.time()
esn = ESN(
    units=chosen_reservoir,
    sr=chosen_spectral_radius,
    input_scaling=chosen_input_scaling,
    ridge=1e-6,
    seed=42
)
esn = esn.fit(X_train_clean.values, y_train_arr)
train_time = time.time() - start
cpu_stop.set()
cpu_thread.join()

Your system has 8 logical CPU cores.



In [29]:
y_pred_test = esn.run(X_test_clean.values)
y_pred_test = (y_pred_test > 0.5).astype(int).ravel()
acc = accuracy_score(y_test.values, y_pred_test)
f1 = f1_score(y_test.values, y_pred_test, average='weighted')

max_cpu_percent = max(cpu_usages)
mean_cpu_percent = np.mean(cpu_usages)
max_cores_used = max_cpu_percent / 100
mean_cores_used = mean_cpu_percent / 100

print(f"Max CPU usage during ESN training: {max_cpu_percent:.2f}% ({max_cores_used:.2f} out of {num_cores} cores fully utilized)")
print(f"Mean CPU usage during ESN training: {mean_cpu_percent:.2f}% ({mean_cores_used:.2f} out of {num_cores} cores on average)")
print(f"Accuracy: {acc:.4f}")
print(f"F1 score: {f1:.4f}")
print(f"Training time: {train_time:.2f} seconds")

Max CPU usage during ESN training: 123.00% (1.23 out of 8 cores fully utilized)
Mean CPU usage during ESN training: 98.48% (0.98 out of 8 cores on average)
Accuracy: 0.9531
F1 score: 0.9482
Training time: 5.68 seconds
